In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808329 sha256=afba51cb1c81a32a5cbf440f7f88f3e7e285b772414c98e4130a43d740eebbb5
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [2]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
data = fetch_movielens(min_rating=5.0)

# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(data['train'], epochs=30, num_threads=2)

# Evaluate the trained model
test_precision = precision_at_k(model, data['test'], k=5).mean()

In [3]:
print("Test precision: %.2f" %(test_precision))

Test precision: 0.05


In [4]:
# BUILD THE MOVIELENS DATASET
import pandas as pd
from lightfm.data import Dataset

ratings = pd.read_csv("/content/ratings.csv")
movies = pd.read_csv("/content/movies.csv")
dataset = Dataset()

In [5]:
dataset.fit(ratings['userId'],
            ratings['movieId'])

In [6]:
users, items = dataset.interactions_shape()
print(f"Users: {users}, items: {items}")

Users: 610, items: 9724


In [7]:
ratings['id_tuple'] = list(zip(ratings['userId'], ratings['movieId']))

In [8]:
(interactions, weights) = dataset.build_interactions((ratings['id_tuple']))

print(repr(interactions))

<610x9724 sparse matrix of type '<class 'numpy.int32'>'
	with 100836 stored elements in COOrdinate format>


In [9]:
import re
#BULD ITEM FEATURES
def split_title_and_yr(df):
    for title in df["title"]:
        release_date = re.findall(r"\(\s*\+?(-?\d+)\s*\)", title) #exp strips brackets
        row_idx = movies.index[df["title"] == title].tolist()[0]
        for m in release_date:
            if int(m) >= 1990:
                df["year"][row_idx] = int(m)  # refactor with a nicer pd function

        df["title"][row_idx] = df["title"][row_idx].split("(")[0].strip()
    return df

In [15]:
genre_list = set()
def get_genres(df):
  for i in range(len(df['genres'])):
    genre_list_per_row = df['genres'][i].split("|")
    for j in genre_list_per_row:
      genre_list.add(j.strip())
  return genre_list

In [16]:
genre_list

set()

In [43]:
def split_genres(df):
  for i in range(len(df['genres'])):
    genre_list_per_row = df['genres'][i].split("|")
    for j in genre_list_per_row:
      df.loc[i, f"genre_{j}"] = True
  return df

In [ ]:
df = movies.copy()
df['year'] = 0
df = split_title_and_yr(df)
genre_list = get_genres(df)

for genre in genre_list:
  df[f'genre_{genre}'] = False

df = split_genres(df)

In [45]:
dataset.fit_partial(items=df['movieId'],
                    item_features = [df["genre_Action"], df['genre_Adventure'], df['Animation']]) #test cols

In [48]:
#BUILD ITEM FEATURES
item_features = dataset.build_item_features(df['movieId'], [df['genre_Action'], df['genre_Adventure'], df['genre_Animation']])

TypeError: object of type 'int' has no len()